In [ ]:
!pip install langchain langchain-core langchain-community chromadb langchain-openai --quiet

In [ ]:
!pip install pypdf


In [ ]:
from pypdf import PdfReader

reader = PdfReader("Área de trabalho/projeto_dieta/Dieta_atual.pdf")
dieta = ""
caloria = ""

for page in reader.pages:
    dieta += page.extract_text() 

reader = PdfReader("Área de trabalho/projeto_dieta/taco_4_edicao_ampliada_e_revisada.pdf")

for page in reader.pages:
    caloria += page.extract_text() 
    
import re
dieta = re.sub(r"\s{2,}", " ", dieta)
caloria = re.sub(r"\s{2,}", " ", caloria)

In [ ]:
from langchain_core.documents import Document
document_dieta = [Document(page_content = dieta, metadata = {"autor": "Nutri do danilo"})]
document_caloria = [Document(page_content = caloria, metadata = {"autor": "Unicamp"})]

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 200,
    length_function = len,
)
chunks_dieta = text_splitter.split_documents(document_dieta)

text_splitter_bigger_docs = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 500,
    length_function = len,
)
chunks_calorias = text_splitter_bigger_docs.split_documents(document_caloria)

In [ ]:
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

OPENAI_API_KEY = ""

db = Chroma.from_documents(
    documents = chunks_dieta + chunks_calorias,
    embedding = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY),
    persist_directory = "db_lite",
)


In [ ]:
query = "o que tomar no café da manha"

docs = db.similarity_search_with_relevance_scores(query, k=5, filter={"autor":"Unicamp"})

for i, d in enumerate(docs, 1):
    print(f"\n--- Resultado {i} ---")
    print("Metadata:", d.metadata)
    print("Texto:", d.page_content[:400])
